# Compute Subtype Weights
- take a subtype
- take the residuals
- compute the weights and store them

In [1]:
# Imports
import os
import numpy as np
import scipy as sp
import pandas as pd
import patsy as pat
import nibabel as nib
from scipy import cluster as scl
from sklearn import linear_model as sln
from sklearn import preprocessing as skp

In [2]:
# Paths
root_p = '/home/surchs/sim_big/PROJECT/abide_hps/'
# Pheno
sample_p = os.path.join(root_p, 'pheno', 'ABIDE_VALIDATE.csv')
subtype_fc_p = os.path.join(root_p, 'subtypes', 'subtypes_fc_TRAIN.npz')
subtype_ct_p = os.path.join(root_p, 'subtypes', 'subtypes_ct_TRAIN.npz')
# Data sd_30mm_residual_nopsm
resid_ct_p = os.path.join(root_p, 'residual', 'ct_30mm_residual_VALIDATE.npy')
resid_fc_p = os.path.join(root_p, 'residual', 'sd_30mm_residual_VALIDATE.npy')
mask_p = os.path.join(root_p, 'mask', 'MIST_mask.nii.gz')
# Out templates
weight_fc_p = os.path.join(root_p, 'weights', 'weights_fc_VALIDATE')
weight_ct_p = os.path.join(root_p, 'weights', 'weights_ct_VALIDATE')

In [3]:
# Load data
sample = pd.read_csv(sample_p)
n_sub = sample.shape[0]
ct = np.load(resid_ct_p)
fc = np.load(resid_fc_p)
mask_i = nib.load(mask_p)
mask = mask_i.get_data().astype(bool)
subtypes_fc = np.load(subtype_fc_p)['subtypes']
subtypes_ct = np.load(subtype_ct_p)['subtypes']
n_subtypes = subtypes_fc.shape[0]

# Compute the CT subtype weights
- 5 subtypes
- distance based
- hierarchical using wards

In [4]:
def corr2_coeff(A,B):
    # Rowwise mean of input arrays & subtract from input arrays themeselves
    A_mA = A - A.mean(1)[:,None]
    B_mB = B - B.mean(1)[:,None]

    # Sum of squares across rows
    ssA = (A_mA**2).sum(1);
    ssB = (B_mB**2).sum(1);

    # Finally get corr coeff
    return np.dot(A_mA,B_mB.T)/np.sqrt(np.dot(ssA[:,None],ssB[None]))

In [5]:
# Compute the weights
weights_ct = corr2_coeff(ct, subtypes_ct)

In [6]:
# Run the FC subtypes
weights_fc = np.zeros((n_sub, n_subtypes, 20))

for sid in range(20):
    # Compute the weights
    weights_fc[..., sid] = corr2_coeff(fc[..., sid], subtypes_fc[..., sid])

In [7]:
# Save everything
np.save(weight_fc_p, weights_fc)
np.save(weight_ct_p, weights_ct)